In [74]:
%use dataframe
%use kandy

In [75]:
enum class StartType {
    COLD,
    WARM
}

data class InvokeResult(
    val functionName: String,
    val requestId: String,
    val duration: Int,
    val billedDuration: Int,
    val initDuration: Int? = null,
    val restoreDuration: Int? = null,
    val billedRestoreDuration: Int? = null,
    val memorySize: Int,
    val maxMemoryUsed: Int,
    val timestamp: Long,
    val startType: StartType
)

# Wczytywanie i preprocessing danych

In [76]:
val rawResultsDataframe = DataFrame.read("results.json")

In [77]:
import org.jetbrains.kotlinx.dataframe.DataFrame

val resultsDataframe = rawResultsDataframe
    .add("startType") {
        if (it["initDuration"] == null && it["restoreDuration"] == null) {
            StartType.WARM
        } else {
            StartType.COLD
        }
    }
    .update(InvokeResult::functionName) {
        it.replace("mte-tested-", "")
            .let { name ->
                if (name.startsWith("java-function-jar-snapstart")) {
                    "Java JVM + SnapStart"
                } else if (name.startsWith("java-function-jar")) {
                    "Java JVM"
                } else if (name.startsWith("java-function-native")) {
                    "Java GraalVM"
                } else if (name.startsWith("kotlin-function-jar-snapstart")) {
                    "Kotlin JVM + SnapStart"
                } else if (name.startsWith("kotlin-function-jar")) {
                    "Kotlin JVM"
                } else if (name.startsWith("kotlin-function-js")) {
                    "Kotlin/JS"
                } else if (name.startsWith("kotlin-function-native")) {
                    "Kotlin/Native"
                } else {
                    throw IllegalArgumentException("Unknown function name: $name")
                }
            }
    }
//    .add("orderValue") {
//        when (it.functionName) {
//            "Java JVM" -> 0
//            "Java JVM + SnapStart" -> 1
//            "Java GraalVM" -> 2
//            "Kotlin JVM" -> 3
//            "Kotlin JVM + SnapStart" -> 4
//            "Kotlin/JS" -> 5
//            "Kotlin/Native" -> 6
//            else -> throw IllegalArgumentException("Unknown function name: ${it.functionName}")
//        }
//    }
    .sortBy(InvokeResult::functionName.name, InvokeResult::memorySize.name)
    .cast<InvokeResult>()

resultsDataframe

requestId,duration,billedDuration,memorySize,maxMemoryUsed,restoreDuration,billedRestoreDuration,timestamp,functionName,initDuration,startType
4b1f11bd-12e5-4c59-a9a5-1e09e92546d5,306.240000,550,128,51,null,null,1747012985806,Java GraalVM,242.880000,COLD
e1fd8f0f-c6c1-4487-8c82-86a9dac73f56,1.240000,2,128,51,null,null,1747012990822,Java GraalVM,null,WARM
00f031b7-0e30-49a9-a72a-f73d3524238b,1.170000,2,128,51,null,null,1747012995851,Java GraalVM,null,WARM
3d5e2c17-f408-4d63-8487-a8961b63d057,1.120000,2,128,51,null,null,1747013000876,Java GraalVM,null,WARM
bdb698f5-ae61-4c7d-b47c-83a5996c0de8,1.280000,2,128,52,null,null,1747013005903,Java GraalVM,null,WARM
a9be3004-c545-471c-b239-66fd42799c93,1.260000,2,128,52,null,null,1747013010927,Java GraalVM,null,WARM
e567f3c4-0c5d-4308-9b5d-da850bb228e0,1.720000,2,128,52,null,null,1747013015952,Java GraalVM,null,WARM
461485ae-cb6c-4486-916a-c8bd17ae43c2,1.300000,2,128,52,null,null,1747013020979,Java GraalVM,null,WARM
78d263a7-7cc0-4c8e-a7fb-8a80b3f41950,1.220000,2,128,52,null,null,1747013026006,Java GraalVM,null,WARM
65c35aa6-4078-4aa7-9189-6106e7f03d36,1.330000,2,128,53,null,null,1747013031032,Java GraalVM,null,WARM


# Koszty funkcji

In [140]:
// Calculation parameters
val coldStartPercentage = 0.025
val rps = 500
val time = 30.days.inWholeSeconds
val gbSecondPrice = 0.0000166667

In [141]:
val averageBilledDuration = resultsDataframe
    .convert(InvokeResult::billedDuration).with { it / 1000.0 } // convert to seconds
    .groupBy("functionName", "startType", "memorySize")
    .aggregate {
        mean(it.billedDuration).into("avgBilledDuration")
    }
    .groupBy("functionName", "memorySize")
    .map { row ->
        val functionName = row.key["functionName"]
        val memorySize = (row.key["memorySize"] as Int)
        val memorySizeGb = memorySize.toDouble() / 1024.0
        val coldStartAvgBilledDuration =
            row.group.first { it.startType == StartType.COLD }["avgBilledDuration"] as Double
        val warmStartAvgBilledDuration =
            row.group.first { it.startType == StartType.WARM }["avgBilledDuration"] as Double

        val coldStartRequestsPerSecond = rps * coldStartPercentage
        val warmStartRequestsPerSecond = (1 - coldStartPercentage) * rps

        val totalDurationSeconds =
            coldStartRequestsPerSecond * time * coldStartAvgBilledDuration + warmStartRequestsPerSecond * time * warmStartAvgBilledDuration

        val calculatedCost = totalDurationSeconds * gbSecondPrice * memorySizeGb

        mapOf(
            "functionName" to functionName,
            "memorySize" to memorySize,
            "calculatedCost" to calculatedCost
        )
    }
    .let { tempMap ->
        dataFrameOf(
            "functionName" to tempMap.map { it["functionName"] },
            "memorySize" to tempMap.map { it["memorySize"] },
            "calculatedCost" to tempMap.map { it["calculatedCost"] },
        )
    }

averageBilledDuration

functionName,memorySize,calculatedCost
Java GraalVM,128,60.811139
Java GraalVM,256,151.746371
Java GraalVM,512,128.300882
Java GraalVM,1024,202.872399
Java GraalVM,2048,397.095194
Java JVM,128,126.631227
Java JVM,256,101.665575
Java JVM,512,109.156065
Java JVM,1024,134.453570
Java JVM,2048,167.118378


In [142]:
import org.jetbrains.letsPlot.scale.scaleXDiscrete

val averageBilledDurationChart = averageBilledDuration
    .sortBy("memorySize")
    .convert("memorySize") { it.toString() }
    .plot {
        // Tworzymy osobny panel dla każdej 'functionName' wzdłuż osi X
        facetGridX(functionName)

        x("memorySize") {
            axis.name = "Rozmiar pamięci [MB]"
        }
        y("calculatedCost") {
            axis.name = "Średni koszt funkcji [USD]"
        }

        bars {
            fillColor("memorySize") {
                legend.type = LegendType.None
            }
        }

        layout {
            size = 1000 to 500
            y.axis {
                breaks(format = "d")
            }
        }
    }

averageBilledDurationChart.save("average-cost.png")
averageBilledDurationChart

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ssy3NJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","128","128","128","128","128","128","256","256","256","256","256","256","256","512","512","512","512","512","512","512","1024","1024","1024","1024","1024","1024","1024","2048","2048","2048","2048","2048","2048","2048"],
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"calculatedCost":[60.81113902899759,126.63122681963945,213.0499029064449,352.00252798323993,505.41802869117856,119.51561181345055,47.92706764265908,151.74637147761166,101.66557547960385,165.70986903663012,319.8611587982342,455.62908523753913,194.34455482353613,43.14259648910207,128.30088160125,109.15606465022809,172.19263278583077,316.4706741488734,454.9018188927272,97.05175082580251,133.57932598164706,202.8723992566239,134.4535702271309,225.65231650749223,336.9513190625808,566.889506427685,158.80850489128457,187.59042489012438,397.09519418880024,167.1183781328919,377.47673950795394,410.5568355177912,908.3664562277113,283.02760687040825,243.0932239598307]
},
"ggsize":{
"width":1000.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rozmiar pamięci [MB]"
},{
"aesthetic":"y",
"name":"Średni koszt funkcji [USD]",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"calculatedCost",
"fill":"memorySize"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"facet":{
"name":"grid",
"x":"functionName",
"x_order":1.0,
"y_order":1.0
}
};
 var plotContainer = document.getElementById("ssy3NJ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

# Metody do wykresów

In [81]:
import org.jetbrains.kotlinx.statistics.kandy.layers.context.BoxplotLayerBuilder
import org.jetbrains.letsPlot.scale.guideLegend
import org.jetbrains.letsPlot.scale.scaleYDiscrete

val defaultBoxplotStyling: BoxplotLayerBuilder<Any?>.() -> Unit = {
    boxes {
        fatten = 0.5
        alpha = 0.6
        borderLine.color(Stat.x) {

            legend {
                type = LegendType.None
            }
        }
    }

    outliers {
        color(Stat.x) {
            legend {
                type = LegendType.None
            }
        }
    }
}

In [82]:
fun DataFrame<Any>.filterByMemorySize(memorySize: Int?) = this
    .let {
        if (memorySize == null) {
            it
        } else {
            it.filter { row ->
                row["memorySize"] == memorySize
            }.also { df ->
                val memorySizeExists = df.rows().toList().size > 0
                if (!memorySizeExists) {
                    throw IllegalArgumentException("Memory size $memorySize doesn't exist")
                }
            }
        }
    }

fun DataFrame<Any>.filterByFunctionName(functionName: String?) = this
    .let {
        if (functionName == null) {
            it
        } else {
            it.filter { row ->
                row["functionName"] == functionName
            }.also { df ->
                val functionNameExists = df.rows().toList().size > 0
                if (!functionNameExists) {
                    throw IllegalArgumentException("Function name $functionName doesn't exist")
                }
            }
        }
    }

In [83]:
val coldStartsDataFrame = resultsDataframe
    .filter { it.startType == StartType.COLD }
    .filter { !it.functionName.contains("snapstart") || it.initDuration == null }
    .add("totalDuration") {
        it.duration.toDouble() + (it.initDuration?.toDouble() ?: 0.0) + (it.restoreDuration?.toDouble() ?: 0.0)
    }

val warmStartsDataframe = resultsDataframe
    .filter { it.startType == StartType.WARM }

In [84]:
fun plotColdStartsBoxplot(memorySize: Int) =
    coldStartsDataFrame
        .filterByMemorySize(memorySize = memorySize)
        .plot {
            layout {
                y.axis {
                    breaks(format = "d")
                }
            }

            boxplot(x = InvokeResult::functionName.name, y = "totalDuration") {
                layout.xAxisLabel = "Rodzaj funkcji"
                layout.yAxisLabel = "Czas wykonania [ms]"
                layout.size = 600 to 600

                defaultBoxplotStyling()
            }
        }
        .also { it.save("cold-start-boxplot-${memorySize}.png") }

In [85]:
fun plotWarmStartsBoxplot(memorySize: Int) =
    warmStartsDataframe
        .filterByMemorySize(memorySize = memorySize)
        .plot {
            layout {
                y.axis {
                    breaks(format = "d")
                }
            }

            boxplot(x = InvokeResult::functionName.name, y = "duration") {
                layout.xAxisLabel = "Rodzaj funkcji"
                layout.yAxisLabel = "Czas wykonania [ms]"
                layout.size = 600 to 600

                outliers.show = false

                defaultBoxplotStyling()
            }
        }
        .also { it.save("warm-start-boxplot-${memorySize}.png") }

In [87]:
fun plotColdStartsBars(memorySize: Int? = null) =
    coldStartsDataFrame
        .filterByMemorySize(memorySize = memorySize)
        .groupBy(InvokeResult::functionName.name)
        .aggregate {
            mean("totalDuration").into("avgDuration")
        }
        .plot {
            layout {
                y.axis {
                    breaks(format = "d")
                }
            }

            bars {
                x("functionName")
                y("avgDuration")

                fillColor("functionName") {
                    legend {
                        name = "Rodzaj funkcji"
                    }
                }
            }

            layout {
                xAxisLabel = "Rodzaj funkcji"
                yAxisLabel = "Średni czas wykonania [ms]"
                size = 1200 to 600
            }
        }
        .also { it.save("avg-cold-start-${memorySize}.png") }


In [88]:
fun plotWarmStartsBars(memorySize: Int? = null) =
    warmStartsDataframe
        .filterByMemorySize(memorySize = memorySize)
        .groupBy(InvokeResult::functionName.name)
        .aggregate {
            mean("duration").into("avgDuration")
        }
        .plot {
            layout {
                y.axis {
                    breaks(format = "d")
                }
            }

            bars {
                x("functionName")
                y("avgDuration")

                fillColor("functionName") {
                    legend {
                        name = "Rodzaj funkcji"
                    }
                }
            }

            layout {
                xAxisLabel = "Rodzaj funkcji"
                yAxisLabel = "Średni czas wykonania [ms]"
                size = 1200 to 600
            }
        }
        .also { it.save("avg-warm-start-${memorySize}.png") }

In [89]:
fun plotColdStartsPointsBasedOnMemory(functionName: String) =
    coldStartsDataFrame
        .filterByFunctionName(functionName = functionName)
        .convert(InvokeResult::memorySize.name).with { it.toString() }
        .sortBy { it[InvokeResult::memorySize.name].convertToInt() }
        .groupBy(InvokeResult::memorySize.name)
        .aggregate {
            mean("totalDuration").into("avgDuration")
        }
        .plot {
            layout {
                y.axis {
                    breaks(format = "d")
                }
            }

            bars {
                x(InvokeResult::memorySize.name)
                y("avgDuration")
            }

            layout {
                xAxisLabel = "Wielkość pamięci funkcji [MB]"
                yAxisLabel = "Średni czas wykonania [ms]"
                size = 1200 to 600
            }
        }
        .also { it.save("cold-start-memory-${functionName.replace(" ", "-").replace("/", "-")}.png") }


In [90]:
fun plotWarmStartsPointsBasedOnMemory(functionName: String) =
    warmStartsDataframe
        .filterByFunctionName(functionName = functionName)
        .convert(InvokeResult::memorySize.name).with { it.toString() }
        .sortBy { it[InvokeResult::memorySize.name].convertToInt() }
        .groupBy(InvokeResult::memorySize.name)
        .aggregate {
            mean("duration").into("avgDuration")
        }
        .plot {
            layout {
                y.axis {
                    breaks(format = "d")
                }
            }

            bars {
                x(InvokeResult::memorySize.name)
                y("avgDuration")
            }

            layout {
                xAxisLabel = "Wielkość pamięci funkcji [MB]"
                yAxisLabel = "Średni czas wykonania [ms]"
                size = 1200 to 600
            }
        }
        .also { it.save("warm-start-memory-${functionName.replace(" ", "-").replace("/", "-")}.png") }

# Analiza zimnych startów

### Boxploty

In [91]:
plotColdStartsBoxplot(memorySize = 128)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ZNBGvW"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[600.22,2735.5,2473.3050000000003,4943.955,5937.41,716.08,323.46],
"min":[549.12,2588.05,2041.38,4613.280000000001,5253.27,587.02,274.61],
"max":[637.04,2909.62,3034.9799999999996,5279.88,6277.700000000001,836.55,370.12],
"lower":[586.49,2694.5175,2351.1000000000004,4841.6,5719.0225,664.84,310.36],
"upper":[611.765,2792.54,2627.64,5024.620000000001,6063.179999999999,735.4,336.35],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java GraalVM","Java GraalVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/JS","Kotlin/JS","Kotlin/JS","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native"],
"y":[522.3399999999999,528.4300000000001,2413.41,2406.87,2428.04,2513.17,2496.7799999999997,2524.09,1910.69,1886.82,1736.75,1760.34,1699.54,1774.2,1795.12,1931.8200000000002,3345.6000000000004,3175.7700000000004,3745.98,4529.429999999999,4875.07,4930.759999999999,350.33,320.98,330.03,319.52,377.15999999999997,378.89,395.53,387.79999999999995,384.77,385.20000000000005]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("ZNBGvW");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 3000 
 
 
 
 
 
 
 4000 
 
 
 
 
 
 
 5000 
 
 
 
 
 
 
 6000 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

In [92]:
plotColdStartsBoxplot(memorySize = 256)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="PvY42o"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[422.0,2196.4,1274.12,2449.1800000000003,3239.765,430.08500000000004,198.43],
"min":[371.71,1966.54,1095.24,2134.06,2697.5299999999997,359.2,153.28],
"max":[440.94,2363.8999999999996,1412.51,2767.2599999999998,3580.42,475.28999999999996,253.88],
"lower":[405.95000000000005,2122.8,1221.59,2367.3199999999997,3081.0024999999996,406.8425,188.70499999999998],
"upper":[429.145,2231.92,1307.6025,2532.8050000000003,3339.2200000000003,438.6925,216.445],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java GraalVM","Java GraalVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native"],
"y":[370.30999999999995,370.0,1942.1,1931.67,1920.69,1893.62,1938.67,1092.43,1080.8,1039.34,2175.6400000000003,1514.48,1050.57,1064.76,1082.42,2242.1,2062.87,1959.93,2053.5699999999997,1967.43,2992.9300000000003,1997.1699999999998,2041.8999999999999,2627.63,2486.72,488.19,279.08,261.26,262.37,263.99,267.49,285.93,259.76]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("PvY42o");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1500 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 2500 
 
 
 
 
 
 
 3000 
 
 
 
 
 
 
 3500 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

In [93]:
plotColdStartsBoxplot(memorySize = 512)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="8LNH9W"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"min":[285.9,1499.15,723.89,1162.75,1508.1,249.99,94.83],
"middle":[343.655,1864.82,887.565,1325.1799999999998,1997.8700000000001,291.69,135.35],
"max":[375.25,2024.9,969.5799999999999,1433.84,2362.06,325.14,192.02],
"upper":[350.87,1923.24,930.5799999999999,1360.3500000000001,2077.5424999999996,300.14,155.32999999999998],
"lower":[313.58500000000004,1741.82,846.435,1272.6125,1816.8874999999998,275.595,128.26999999999998],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/JS","Kotlin/JS","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native"],
"y":[1466.56,708.99,704.0,698.78,1841.94,668.63,1110.69,1569.3999999999999,1783.03,1407.8999999999999,1413.84,1390.0600000000002,1350.8400000000001,1297.35,199.52999999999997,210.58,227.39999999999998,223.76,256.18,206.21,214.86,198.56,195.99]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("8LNH9W");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1500 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

In [94]:
plotColdStartsBoxplot(memorySize = 1024)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3uZCSJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[307.39,1681.09,719.9649999999999,760.33,1417.63,230.375,108.99],
"min":[237.31,1462.81,593.07,692.01,923.4000000000001,185.62,81.53],
"max":[318.92,1827.65,819.5699999999999,826.1299999999999,1682.8400000000001,268.62,182.99],
"lower":[279.835,1549.17,686.0649999999999,733.315,1256.93,215.3875,102.41],
"upper":[313.47,1729.82,748.56,781.96,1528.7050000000002,239.27,135.19],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java GraalVM","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin/Native"],
"y":[392.29,588.74,589.2,1731.0,589.07,1703.5900000000001,504.57,863.51,910.4200000000001,503.83000000000004,512.36,813.39,733.77,817.66,823.31,753.05,778.31,189.1]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("3uZCSJ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1600 
 
 
 
 
 
 
 1800 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

In [95]:
plotColdStartsBoxplot(memorySize = 2048)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="0axqME"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"min":[243.15,1372.51,544.07,445.59000000000003,885.87,173.82,70.42],
"middle":[297.53999999999996,1472.77,712.26,493.75,1351.335,203.985,99.505],
"max":[325.21,1539.26,849.3000000000001,534.31,1618.2,223.1,145.47],
"upper":[301.57,1490.06,751.11,504.87,1458.2375,210.03,117.0175],
"lower":[260.8575,1437.69,668.05,481.14,1190.0025,193.66,92.615],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native"],
"y":[1131.29,1093.91,1341.0,1586.88,1357.02,1347.3999999999999,429.85,463.34,539.36,525.77,1512.11,878.43,944.52,953.5,998.5999999999999,912.54,976.91,1527.8999999999999,1375.75,442.07,592.91,443.46000000000004,858.73,607.6800000000001,581.87,600.1600000000001,551.74,552.8399999999999,665.26,687.1099999999999,637.9,767.75,758.6800000000001,495.37,497.71000000000004,159.04,154.01,156.8,182.46,167.2,159.95]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("0axqME");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1600 
 
 
 
 
 
 
 
 
 Czas wykonan

### Srednie

In [96]:
plotColdStartsBars()

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gA6EQo"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[386.5791020408165,1962.9243410852723,1328.676120906801,1941.513753213367,2721.9967118644095,347.86425339366497,168.52720095693775]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("gA6EQo");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1600 
 
 
 
 
 
 
 1800 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 2200 
 
 
 
 
 
 
 2400 
 
 
 
 
 
 
 2600 
 
 
 
 
 
 
 2800 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [97]:
plotColdStartsBars(memorySize = 128)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="07ZGYG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[596.0436734693878,2725.8396153846156,2465.925357142857,4926.535000000002,5853.267666666668,679.1654545454543,328.2998039215686]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("07ZGYG");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1500 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 2500 
 
 
 
 
 
 
 3000 
 
 
 
 
 
 
 3500 
 
 
 
 
 
 
 4000 
 
 
 
 
 
 
 4500 
 
 
 
 
 
 
 5000 
 
 
 
 
 
 
 5500 
 
 
 
 
 
 
 6000 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [98]:
plotColdStartsBars(memorySize = 256)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="wMNWsN"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[416.56020408163266,2165.0827450980387,1287.4543103448277,2426.347826086957,3185.3465517241384,424.45078124999986,204.4944761904763]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("wMNWsN");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1500 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 2500 
 
 
 
 
 
 
 3000 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [99]:
plotColdStartsBars(memorySize = 512)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="4v2Cjg"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[335.18104166666666,1826.7390196078427,901.4643103448278,1319.8440384615385,1915.499827586207,286.6268235294117,142.86266666666657]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("4v2Cjg");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1600 
 
 
 
 
 
 
 1800 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [100]:
plotColdStartsBars(memorySize = 1024)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="XWh1Qf"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[299.286530612245,1651.756470588235,745.4489655172415,754.1882795698925,1346.7762295081964,227.63313725490184,119.18392156862745]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("XWh1Qf");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1600 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [101]:
plotColdStartsBars(memorySize = 2048)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="PyC9t5"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[286.8112,1450.345849056603,730.6978378378377,504.25568627451,1272.2505172413787,201.25472222222223,105.19056603773582]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("PyC9t5");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1100 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1300 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1500 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

# Analiza ciepłych startów

In [102]:
plotWarmStartsBoxplot(128)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="WDORSV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[1.13,14.55,35.86,10.32,44.69,2.5599999999999996,7.08],
"min":[0.99,1.9,2.4,1.83,6.65,1.56,2.09],
"max":[1.57,37.56,75.27,30.61,137.37,68.55,27.62],
"lower":[1.08,8.9575,27.770000000000003,2.01,30.5075,1.8325,2.285],
"upper":[1.28,22.96,50.705,14.65,75.6975,30.665,14.185],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
}]
};
 var plotContainer = document.getElementById("WDORSV");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

In [103]:
plotWarmStartsBoxplot(256)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iI5H3h"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"min":[0.92,1.8,2.28,1.76,2.0,2.0,1.82],
"middle":[1.25,2.16,11.95,2.13,13.41,10.24,2.06],
"max":[77.95,2.97,25.55,5.62,43.3,59.54,2.37],
"upper":[38.77,2.535,15.445,3.465,22.085,33.56999999999999,2.215],
"lower":[1.09,2.01,2.565,1.96,6.625,2.335,1.98],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
}]
};
 var plotContainer = document.getElementById("iI5H3h");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

In [104]:
plotWarmStartsBoxplot(512)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="24CV6l"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[1.3,2.37,2.8600000000000003,2.03,2.935,1.9649999999999999,6.8],
"min":[1.07,2.09,2.18,1.64,2.0,1.56,2.03],
"max":[1.56,2.96,4.13,3.58,6.8,4.46,22.64],
"lower":[1.22,2.215,2.43,1.8450000000000002,2.3850000000000002,1.8,2.35],
"upper":[1.3675000000000002,2.615,3.8675,2.5650000000000004,4.257499999999999,3.0375,14.02],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
}]
};
 var plotContainer = document.getElementById("24CV6l");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

In [105]:
plotWarmStartsBoxplot(1024)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="yo5w5T"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[1.01,2.42,2.92,2.75,3.09,3.04,6.5649999999999995],
"min":[0.76,2.04,2.34,1.78,2.1,1.62,2.07],
"max":[1.42,3.17,5.75,3.97,8.44,10.39,8.42],
"lower":[0.91,2.25,2.525,2.3425000000000002,2.455,1.9649999999999999,2.515],
"upper":[1.12,2.665,4.2325,3.065,4.869999999999999,6.0649999999999995,6.9725],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
}]
};
 var plotContainer = document.getElementById("yo5w5T");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

In [106]:
plotWarmStartsBoxplot(2048)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="d0ssAN"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":600.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[1.07,1.44,2.8,2.12,2.885,4.02,2.24],
"min":[0.83,1.08,2.3,1.78,2.06,2.0,2.06],
"max":[1.34,3.68,4.98,3.68,5.93,12.15,2.7],
"lower":[1.01,1.28,2.535,1.96,2.385,2.33,2.15],
"upper":[1.1624999999999999,2.28,3.59,2.68,3.845,6.625,2.375],
"x":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
}]
};
 var plotContainer = document.getElementById("d0ssAN");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji

### Srednie

In [107]:
plotWarmStartsBars()

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="TzhkUB"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[6.688536155202823,4.534970760233926,11.689348837209305,4.493966480446929,16.07353488372092,17.273528183716067,5.9841911764705875]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("TzhkUB");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [108]:
plotWarmStartsBars(memorySize = 128)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="E71hWz"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[7.086666666666666,17.885416666666668,37.360975609756096,13.559591836734695,52.918571428571425,32.1531707317073,9.194081632653061]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("E71hWz");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [109]:
plotWarmStartsBars(memorySize = 256)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="zYqHCu"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[17.47895705521472,3.65591836734694,10.307727272727274,4.701428571428572,15.874545454545453,29.977346938775508,2.371224489795918]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("zYqHCu");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 22 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 26 
 
 
 
 
 
 
 28 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [110]:
plotWarmStartsBars(memorySize = 512)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MRj1ri"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[2.8734615384615387,2.5563265306122447,5.1454545454545455,2.410408163265306,5.129545454545456,5.934999999999995,8.449831932773108]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("MRj1ri");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [111]:
plotWarmStartsBars(memorySize = 1024)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="YXkftM"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[1.4759602649006627,2.4767346938775514,3.7425,2.886049382716049,3.885121951219512,5.659591836734693,5.30774647887324]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("YXkftM");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [112]:
plotWarmStartsBars(memorySize = 2048)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="W6CTNO"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"avgDuration":[1.3974666666666666,1.8142857142857145,3.257142857142858,2.6204081632653056,3.403636363636362,5.257346938775511,2.359591836734694]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"avgDuration",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("W6CTNO");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 Rodzaj funkcji 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native

In [113]:
import org.jetbrains.letsPlot.core.spec.back.transform.bistro.util.scale

warmStartsDataframe
    .convert(InvokeResult::memorySize.name).with { it.toString() }
    .groupBy(InvokeResult::functionName.name, InvokeResult::memorySize.name)
    .aggregate {
        mean("duration").into("avgDuration")
    }
    .plot {
        facetGridX(functionName)

        x("memorySize") {
            axis.name = "Rozmiar pamięci [MB]"
        }
        y("avgDuration") {
            axis.name = "Średni czas wykonania [ms]"
        }

        bars {
            fillColor("memorySize") {
                legend.type = LegendType.None
            }
        }

        layout {
            y.axis {
                breaks(format = "d")
            }
            size = 1000 to 500
        }
    }
    .also { it.save("avg-warm-start.png") }


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="6RzAHC"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048","128","256","512","1024","2048","128","256","512","1024","2048","128","256","512","1024","2048","128","256","512","1024","2048","128","256","512","1024","2048","128","256","512","1024","2048"],
"functionName":["Java GraalVM","Java GraalVM","Java GraalVM","Java GraalVM","Java GraalVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/JS","Kotlin/JS","Kotlin/JS","Kotlin/JS","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native","Kotlin/Native"],
"avgDuration":[7.086666666666666,17.47895705521472,2.8734615384615387,1.4759602649006627,1.3974666666666666,17.885416666666668,3.65591836734694,2.5563265306122447,2.4767346938775514,1.8142857142857145,37.360975609756096,10.307727272727274,5.1454545454545455,3.7425,3.257142857142858,13.559591836734695,4.701428571428572,2.410408163265306,2.886049382716049,2.6204081632653056,52.918571428571425,15.874545454545453,5.129545454545456,3.885121951219512,3.403636363636362,32.1531707317073,29.977346938775508,5.934999999999995,5.659591836734693,5.257346938775511,9.194081632653061,2.371224489795918,8.449831932773108,5.30774647887324,2.359591836734694]
},
"ggsize":{
"width":1000.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rozmiar pamięci [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration",
"fill":"memorySize"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"facet":{
"name":"grid",
"x":"functionName",
"x_order":1.0,
"y_order":1.0
}
};
 var plotContainer = document.getElementById("6RzAHC");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 

In [114]:
coldStartsDataFrame
    .filter { it.memorySize <= 256 }
    .sortBy(InvokeResult::functionName.name, InvokeResult::memorySize.name)
    .convert(InvokeResult::memorySize.name).with { it.toString() }
    .groupBy(InvokeResult::functionName.name, InvokeResult::memorySize.name)
    .aggregate {
        mean("totalDuration").into("avgDuration")
    }
    .plot {
        facetGridX(functionName)

        x("memorySize") {
            axis.name = "Rozmiar pamięci [MB]"
        }
        y("avgDuration") {
            axis.name = "Średni czas wykonania [ms]"
        }

        bars {
            fillColor("memorySize") {
                legend.type = LegendType.None
            }
        }

        layout {
            y.axis {
                breaks(format = "d")
            }
            size = 1000 to 500
        }
    }
    .also { it.save("avg-cold-start-128-256.png") }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="kPwR2A"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","128","256","128","256","128","256","128","256","128","256","128","256"],
"functionName":["Java GraalVM","Java GraalVM","Java JVM","Java JVM","Java JVM + SnapStart","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/JS","Kotlin/Native","Kotlin/Native"],
"avgDuration":[596.0436734693878,416.56020408163266,2725.8396153846156,2165.0827450980387,2465.925357142857,1287.4543103448277,4926.535000000002,2426.347826086957,5853.267666666668,3185.3465517241384,679.1654545454543,424.45078124999986,328.2998039215686,204.4944761904763]
},
"ggsize":{
"width":1000.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rozmiar pamięci [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration",
"fill":"memorySize"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"facet":{
"name":"grid",
"x":"functionName",
"x_order":1.0,
"y_order":1.0
}
};
 var plotContainer = document.getElementById("kPwR2A");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 3000 
 
 
 
 
 
 
 4000 
 
 
 
 
 
 
 5000 
 
 
 
 
 
 
 6000 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rozmiar pamięci [MB]

In [115]:
coldStartsDataFrame
    .filter { it.memorySize > 256 }
    .sortBy(InvokeResult::functionName.name, InvokeResult::memorySize.name)
    .convert(InvokeResult::memorySize.name).with { it.toString() }
    .groupBy(InvokeResult::functionName.name, InvokeResult::memorySize.name)
    .aggregate {
        mean("totalDuration").into("avgDuration")
    }
    .plot {
        facetGridX(functionName)

        x("memorySize") {
            axis.name = "Rozmiar pamięci [MB]"
        }
        y("avgDuration") {
            axis.name = "Średni czas wykonania [ms]"
        }

        bars {
            fillColor("memorySize") {
                legend.type = LegendType.None
            }
        }

        layout {
            y.axis {
                breaks(format = "d")
            }
            size = 1000 to 500
        }
    }
    .also { it.save("avg-cold-start-512-2048.png") }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UYmIUp"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["512","1024","2048","512","1024","2048","512","1024","2048","512","1024","2048","512","1024","2048","512","1024","2048","512","1024","2048"],
"functionName":["Java GraalVM","Java GraalVM","Java GraalVM","Java JVM","Java JVM","Java JVM","Java JVM + SnapStart","Java JVM + SnapStart","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/JS","Kotlin/JS","Kotlin/Native","Kotlin/Native","Kotlin/Native"],
"avgDuration":[335.18104166666666,299.286530612245,286.8112,1826.7390196078427,1651.756470588235,1450.345849056603,901.4643103448278,745.4489655172415,730.6978378378377,1319.8440384615385,754.1882795698925,504.25568627451,1915.499827586207,1346.7762295081964,1272.2505172413787,286.6268235294117,227.63313725490184,201.25472222222223,142.86266666666657,119.18392156862745,105.19056603773582]
},
"ggsize":{
"width":1000.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rozmiar pamięci [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"guide":"none"
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration",
"fill":"memorySize"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"facet":{
"name":"grid",
"x":"functionName",
"x_order":1.0,
"y_order":1.0
}
};
 var plotContainer = document.getElementById("UYmIUp");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1600 
 
 
 
 
 
 
 1800 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Rozmiar pamięci [MB]

# Analiza metod w zaleznosci od rozmiaru pamieci

### Zimne starty

In [116]:
plotColdStartsPointsBasedOnMemory(functionName = "Java JVM")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="dqZIuj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[2725.8396153846156,2165.0827450980387,1826.7390196078427,1651.756470588235,1450.345849056603]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("dqZIuj");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1600 
 
 
 
 
 
 
 1800 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 2200 
 
 
 
 
 
 
 2400 
 
 
 
 
 
 
 2600 
 
 
 
 
 
 
 2800 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [117]:
plotColdStartsPointsBasedOnMemory(functionName = "Java GraalVM")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="EsfFsA"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[596.0436734693878,416.56020408163266,335.18104166666666,299.286530612245,286.8112]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("EsfFsA");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 350 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 450 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 550 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [118]:
plotColdStartsPointsBasedOnMemory(functionName = "Kotlin JVM")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="qtYqeb"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[4926.535000000002,2426.347826086957,1319.8440384615385,754.1882795698925,504.25568627451]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("qtYqeb");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1500 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 2500 
 
 
 
 
 
 
 3000 
 
 
 
 
 
 
 3500 
 
 
 
 
 
 
 4000 
 
 
 
 
 
 
 4500 
 
 
 
 
 
 
 5000 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [119]:
plotColdStartsPointsBasedOnMemory(functionName = "Kotlin/Native")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="6cBayE"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[328.2998039215686,204.4944761904763,142.86266666666657,119.18392156862745,105.19056603773582]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("6cBayE");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [120]:
plotColdStartsPointsBasedOnMemory(functionName = "Kotlin/JS")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="VlnkL0"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[679.1654545454543,424.45078124999986,286.6268235294117,227.63313725490184,201.25472222222223]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("VlnkL0");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 350 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 450 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 550 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 650 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [121]:
plotColdStartsPointsBasedOnMemory(functionName = "Java JVM + SnapStart")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="lJg1Fu"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[2465.925357142857,1287.4543103448277,901.4643103448278,745.4489655172415,730.6978378378377]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("lJg1Fu");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1200 
 
 
 
 
 
 
 1400 
 
 
 
 
 
 
 1600 
 
 
 
 
 
 
 1800 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 2200 
 
 
 
 
 
 
 2400 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [122]:
plotColdStartsPointsBasedOnMemory(functionName = "Kotlin JVM + SnapStart")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="zwyayM"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[5853.267666666668,3185.3465517241384,1915.499827586207,1346.7762295081964,1272.2505172413787]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("zwyayM");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1000 
 
 
 
 
 
 
 1500 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 2500 
 
 
 
 
 
 
 3000 
 
 
 
 
 
 
 3500 
 
 
 
 
 
 
 4000 
 
 
 
 
 
 
 4500 
 
 
 
 
 
 
 5000 
 
 
 
 
 
 
 5500 
 
 
 
 
 
 
 6000 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

### Cieple starty

In [123]:
plotWarmStartsPointsBasedOnMemory(functionName = "Java JVM")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="lHmyJ4"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[17.885416666666668,3.65591836734694,2.5563265306122447,2.4767346938775514,1.8142857142857145]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("lHmyJ4");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [124]:
plotWarmStartsPointsBasedOnMemory(functionName = "Java GraalVM")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="IDOqfo"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[7.086666666666666,17.47895705521472,2.8734615384615387,1.4759602649006627,1.3974666666666666]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("IDOqfo");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [125]:
plotWarmStartsPointsBasedOnMemory(functionName = "Kotlin JVM")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="CSwKRm"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[13.559591836734695,4.701428571428572,2.410408163265306,2.886049382716049,2.6204081632653056]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("CSwKRm");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 11 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 13 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [126]:
plotWarmStartsPointsBasedOnMemory(functionName = "Kotlin/Native")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="emrpOY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[9.194081632653061,2.371224489795918,8.449831932773108,5.30774647887324,2.359591836734694]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("emrpOY");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [127]:
plotWarmStartsPointsBasedOnMemory(functionName = "Kotlin/JS")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vSZva6"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[32.1531707317073,29.977346938775508,5.934999999999995,5.659591836734693,5.257346938775511]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("vSZva6");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [128]:
plotWarmStartsPointsBasedOnMemory(functionName = "Java JVM + SnapStart")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ADgYB5"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[37.360975609756096,10.307727272727274,5.1454545454545455,3.7425,3.257142857142858]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("ADgYB5");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

In [129]:
plotWarmStartsPointsBasedOnMemory(functionName = "Kotlin JVM + SnapStart")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="udMRcy"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","256","512","1024","2048"],
"avgDuration":[52.918571428571425,15.874545454545453,5.129545454545456,3.885121951219512,3.403636363636362]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"format":"d",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Wielkość pamięci funkcji [MB]"
},{
"aesthetic":"y",
"name":"Średni czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"avgDuration"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("udMRcy");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 
 Średni czas wykonania [ms] 
 
 
 
 
 Wielkość pamięci funkcji [MB]

# Wydajność funkcji

In [130]:
val warmStartWeight = 0.975
val coldStartWeight = 1.0 - warmStartWeight

val averageColdStarts = coldStartsDataFrame
    .groupBy("functionName", "memorySize")
    .aggregate { mean(it.totalDuration).into("coldStartDuration") }

val averageWarmStarts = warmStartsDataframe
    .groupBy("functionName", "memorySize")
    .aggregate { mean(it.duration).into("warmStartDuration") }

val averageStarts = averageColdStarts
    .join(averageWarmStarts)
    .add("performanceIndex") { row ->
        val coldDuration = row["coldStartDuration"] as Double
        val warmDuration = row["warmStartDuration"] as Double
        1000.0 / (coldDuration * coldStartWeight + warmDuration * warmStartWeight)
    }
    .convert("memorySize") { it.toString() }
    .sortBy { it["memorySize"].convertToInt() }


averageStarts

functionName,memorySize,coldStartDuration,warmStartDuration,performanceIndex
Java GraalVM,128,596.043673,7.086667,45.849283
Java JVM,128,2725.839615,17.885417,11.684390
Java JVM + SnapStart,128,2465.925357,37.360976,10.196270
Kotlin JVM,128,4926.535000,13.559592,7.332240
Kotlin JVM + SnapStart,128,5853.267667,52.918571,5.052360
Kotlin/JS,128,679.165455,32.153171,20.691734
Kotlin/Native,128,328.299804,9.194082,58.235269
Java GraalVM,256,416.560204,17.478957,36.421927
Java JVM,256,2165.082745,3.655918,17.333549
Java JVM + SnapStart,256,1287.454310,10.307727,23.676265


In [131]:
val wwfPlot = averageStarts.plot {
    // Tworzymy osobny panel dla każdej 'functionName' wzdłuż osi X
    facetGridX(functionName)

    x("memorySize") {
        axis.name = "Rozmiar pamięci [MB]"
    }
    y("performanceIndex") {
        axis.name = "Współczynnik wydajności funkcji (WWF)"
    }

    bars {
        fillColor("memorySize") {
            legend.type = LegendType.None
        }
    }

    layout {
        size = 1000 to 500 // Może wymagać dostosowania w zależności od liczby funkcji
    }
}

wwfPlot.save("wwf.png")
wwfPlot


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="FAN18h"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"memorySize":["128","128","128","128","128","128","128","256","256","256","256","256","256","256","512","512","512","512","512","512","512","1024","1024","1024","1024","1024","1024","1024","2048","2048","2048","2048","2048","2048","2048"],
"functionName":["Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native","Java GraalVM","Java JVM","Java JVM + SnapStart","Kotlin JVM","Kotlin JVM + SnapStart","Kotlin/JS","Kotlin/Native"],
"performanceIndex":[45.849283113709,11.684389910675359,10.196269506063635,7.332239620059796,5.052360164639812,20.69173383814435,58.23526862095432,36.42192703431692,17.333549252433972,23.67626485620675,15.327411478427118,10.513992768927304,25.100916479804006,134.69272809079598,89.43623033742142,20.763734222997154,36.29312747393586,28.291545228544404,18.907593906135812,77.20639119047827,84.67290955838679,112.09223547242709,22.878725684092764,44.872907643529196,46.14971723649204,26.69699470884457,89.21457782980792,122.62940714989213,117.19468733043385,26.2967072003946,46.63491711093625,65.95744770701393,28.469906199133533,98.45154115660215,202.8246911297038]
},
"ggsize":{
"width":1000.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rozmiar pamięci [MB]"
},{
"aesthetic":"y",
"name":"Współczynnik wydajności funkcji (WWF)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"memorySize",
"y":"performanceIndex",
"fill":"memorySize"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"facet":{
"name":"grid",
"x":"functionName",
"x_order":1.0,
"y_order":1.0
}
};
 var plotContainer = document.getElementById("FAN18h");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 160 
 
 
 
 
 
 
 180 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM + SnapStart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 128 
 
 
 
 
 
 
 
 
 256 
 
 
 
 
 
 
 
 
 512 
 
 
 
 
 
 
 
 
 1024 
 
 
 
 
 
 
 
 
 2048 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


# Inne czynniki

In [132]:
val buildTimeMap = mapOf(
    "Java JVM" to listOf(54, 51, 47),
    "Java GraalVM" to listOf(151, 154, 154),
    "Kotlin JVM" to listOf(51, 50, 50),
    "Kotlin/Native" to listOf(151, 154, 156),
    "Kotlin/JS" to listOf(57, 60, 64),
)


val functionNames = mutableListOf<String>()
val buildTimes = mutableListOf<Int>()

buildTimeMap.forEach { (functionName, times) ->
    times.forEach { buildTime ->
        functionNames.add(functionName)
        buildTimes.add(buildTime)
    }
}

val buildTimesDataFrame = dataFrameOf(
    "functionName" to functionNames,
    "buildTime" to buildTimes
)

buildTimesDataFrame

functionName,buildTime
Java JVM,54
Java JVM,51
Java JVM,47
Java GraalVM,151
Java GraalVM,154
Java GraalVM,154
Kotlin JVM,51
Kotlin JVM,50
Kotlin JVM,50
Kotlin/Native,151


In [133]:
buildTimesDataFrame
    .groupBy("functionName")
    .aggregate {
        mean("buildTime").into("buildTime")
    }
    .sortBy("functionName")
    .plot {
        x("functionName") {
            axis.name = "Rodzaj funkcji"
        }
        y("buildTime") {
            axis.name = "Czas budowy artefaktu [s]"
        }
        bars {
            fillColor("functionName") {
                legend.type = LegendType.None
            }
        }
        layout {
            size = 1000 to 500
        }
    }.also {
        it.save("avg-build-times.png")
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="dqa0Z0"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Kotlin JVM","Kotlin/JS","Kotlin/Native"],
"buildTime":[153.0,50.666666666666664,50.333333333333336,60.333333333333336,153.66666666666666]
},
"ggsize":{
"width":1000.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Czas budowy artefaktu [s]",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"buildTime",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("dqa0Z0");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 160 
 
 
 
 
 
 
 
 
 Czas budowy artefaktu [s] 
 
 
 
 
 Rodzaj funkcji

In [134]:
val artifactSize = dataFrameOf(
    "functionName" to listOf("Java JVM", "Java GraalVM", "Kotlin JVM", "Kotlin/Native", "Kotlin/JS"),
    "artifactSize" to listOf(8.6, 14.4, 4.4, 6.2, 1.5)
).sortBy("functionName")

artifactSize.plot {
    x("functionName") {
        axis.name = "Rodzaj funkcji"
    }
    y("artifactSize") {
        axis.name = "Rozmiar artefaktu [MB]"
    }
    bars {
        fillColor("functionName") {
            legend.type = LegendType.None
        }
    }
    layout {
        size = 1000 to 500
    }
}.also {
    it.save("artifact-size.png")
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vVz3Wc"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"functionName":["Java GraalVM","Java JVM","Kotlin JVM","Kotlin/JS","Kotlin/Native"],
"artifactSize":[14.4,8.6,4.4,1.5,6.2]
},
"ggsize":{
"width":1000.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rodzaj funkcji"
},{
"aesthetic":"y",
"name":"Rozmiar artefaktu [MB]",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"guide":"none"
}],
"layers":[{
"mapping":{
"x":"functionName",
"y":"artifactSize",
"fill":"functionName"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("vVz3Wc");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java GraalVM 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin/JS 
 
 
 
 
 
 
 
 
 Kotlin/Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 Rozmiar artefaktu [MB] 
 
 
 
 
 Rodzaj funkcji

# Roznice zimny start

In [135]:
val baselineFunctionName = "Java JVM"

val coldStartsMap = coldStartsDataFrame
    .groupBy("functionName", "memorySize")
    .aggregate {
        mean("totalDuration").into("avgDuration")
    }
    .toMap()

val functionNames = coldStartsMap["functionName"]!!
val memorySize = coldStartsMap["memorySize"]!!
val avgDuration = coldStartsMap["avgDuration"]!!

val baselineAvgDuration = mutableMapOf<Int, Double>()

val differences = functionNames.zip(memorySize).zip(avgDuration)
    .map {
        Triple(it.first.first as String, it.first.second as Int, it.second as Double)
    }
    .also {
        it.forEach { (functionName, memorySize, avgDuration) ->
            if (functionName == baselineFunctionName) {
                baselineAvgDuration[memorySize] = avgDuration
            }
        }
    }
    .map {
        val (functionName, memorySize, avgDuration) = it
        val baselineAvgDuration = baselineAvgDuration[memorySize]!!
        val percentageDifference = (avgDuration - baselineAvgDuration) / baselineAvgDuration

        round(percentageDifference * 100).toInt()
    }

dataFrameOf(
    "functionName" to functionNames,
    "memorySize" to memorySize,
    "avgDuration" to avgDuration.map { round(it as Double) },
    "differences" to differences
)

functionName,memorySize,avgDuration,differences
Java GraalVM,128,596.000000,-78
Java GraalVM,256,417.000000,-81
Java GraalVM,512,335.000000,-82
Java GraalVM,1024,299.000000,-82
Java GraalVM,2048,287.000000,-80
Java JVM,128,2726.000000,0
Java JVM,256,2165.000000,0
Java JVM,512,1827.000000,0
Java JVM,1024,1652.000000,0
Java JVM,2048,1450.000000,0


In [136]:
val baselineFunctionName = "Java JVM"

val warmStartsMap = warmStartsDataframe
    .groupBy("functionName", "memorySize")
    .aggregate {
        mean("duration").into("avgDuration")
    }
    .convert("avgDuration") { ((it as Double) * 100).toInt() / 100.0 }
    .toMap()

val functionNames = warmStartsMap["functionName"]!!
val memorySize = warmStartsMap["memorySize"]!!
val avgDuration = warmStartsMap["avgDuration"]!!

val baselineAvgDuration = mutableMapOf<Int, Double>()

val differences = functionNames.zip(memorySize).zip(avgDuration)
    .map {
        Triple(it.first.first as String, it.first.second as Int, it.second as Double)
    }
    .also {
        it.forEach { (functionName, memorySize, avgDuration) ->
            if (functionName == baselineFunctionName) {
                baselineAvgDuration[memorySize] = avgDuration
            }
        }
    }
    .map {
        val (functionName, memorySize, avgDuration) = it
        val baselineAvgDuration = baselineAvgDuration[memorySize]!!
        val percentageDifference = (avgDuration - baselineAvgDuration) / baselineAvgDuration

        round(percentageDifference * 100).toInt()
    }

dataFrameOf(
    "functionName" to functionNames,
    "memorySize" to memorySize,
    "avgDuration" to avgDuration,
    "differences" to differences
)

functionName,memorySize,avgDuration,differences
Java GraalVM,128,7.080000,-60
Java GraalVM,256,17.470000,379
Java GraalVM,512,2.870000,13
Java GraalVM,1024,1.470000,-40
Java GraalVM,2048,1.390000,-23
Java JVM,128,17.880000,0
Java JVM,256,3.650000,0
Java JVM,512,2.550000,0
Java JVM,1024,2.470000,0
Java JVM,2048,1.810000,0
